In [1]:
# -*- coding: utf-8 -*-

'''
@Author   :   Corley Tang
@contact  :   cutercorleytd@gmail.com
@Github   :   https://github.com/corleytd
@Time     :   2023-12-01 20:50
@Project  :   Hands-on Crawler with Python-coroutine
协程
'''

# 导入所需的库
import asyncio
import random
import time
from collections import namedtuple
from functools import wraps
from inspect import getgeneratorstate

# 1.生成器与协程的关系

In [2]:
# 定义协程：使用生成器函数定义，yield关键字
def simple_coroutine():
    print('start')
    x = yield  # 接收调用端传入的数据
    print(f'x={x}')
    print('end')


coro = simple_coroutine()
coro  # 与创建生成器方式一样，调用函数获得生成器对象

<generator object simple_coroutine at 0x000001159CBC4190>

In [3]:
next(coro)  # 第1次使用，是调用next函数启动生成器，预激（prime）协程，如果生成器没有启动，就不会在yield语句处暂停，就无法发送数据，等价于coro.send(None)

start


In [4]:
# 第2次使用，协议定义体中的yield表达式会计算出3，执行后协程会恢复、直到运行到下一个yield表达式，或者终止
coro.send(3)  # 控制权流动到协程定义体末尾，导致生成器抛出StopIteration异常

x=3
end


StopIteration: 

In [5]:
coro = simple_coroutine()
coro.send(3)  # 仅当协程处于暂停状态时才能调用send方法，刚创建、还未激活的协程（状态为GEN_CREATED）不能调用send方法

TypeError: can't send non-None value to a just-started generator

# 2.产生多值的协程

In [6]:
def multi_value_coroutine(a):
    print(f'start: a={a}')
    b = yield a  # 协程在yield关键字所在的位置暂停执行，接收调用端传入的数据
    print(f'received: b={b}')
    c = yield a + b  # 接收调用端传入的数据
    print(f'received: c={c}')  # 在赋值语句中，=右边的代码在赋值之前执行，本行代码需要等到客户端调用协程时才会设定b的值，同时会先计算a+b的值
    print('end')


coro = multi_value_coroutine(1)  # 获取协程对象
coro

<generator object multi_value_coroutine at 0x000001159E1B6660>

In [7]:
getgeneratorstate(coro)  # 查看协程状态，处于GEN_CREATED状态、协程未启动

'GEN_CREATED'

In [8]:
coro.send(None)  # 协程向前执行到第1个yield表达式，输出start: a=1，同时返回a的值（1），并且暂停、等待为b赋值

start: a=1


1

In [9]:
getgeneratorstate(coro)  # 处于GEN_SUSPENDED状态、协程在yield处暂停

'GEN_SUSPENDED'

In [10]:
coro.send(2)  # 将2传给yield表达式，将2赋值给b，输出received: b=2，同时返回a+b的值（3），暂停、等待为c赋值

received: b=2


3

In [11]:
getgeneratorstate(coro)

'GEN_SUSPENDED'

In [12]:
coro.send(3)  # 将3传给yield表达式，将3赋值给c，输出received: c=3，协程恢复、继续执行到下一个yield表达式，输出end，协程终止，导致生成器抛出StopIteration异常

received: c=3
end


StopIteration: 

In [13]:
getgeneratorstate(coro)  # 处于GEN_CLOSED状态、协程终止

'GEN_CLOSED'

# 3.使用协程计算移动平均值

In [14]:
# 使用协程计算移动平均值
def make_avg():
    total = 0
    count = 0
    avg = None
    while True:  # 无限循环保证协程可以不断接收值，同时计算均值并返回
        num = yield avg  # yield表达式用于：暂停协程，把结果返回给调用方；接收调用方传入的值，并进行计算
        total += num
        count += 1
        avg = total / count


avg_coro = make_avg()  # 创建协程对象
avg_coro

<generator object make_avg at 0x000001159CC096D0>

In [15]:
avg_coro.send(None)  # 预激协程，协程会向前执行到yield表达式，返回avg变量的初始值（None），因此不会出现在控制台中，同时在yield表达式出暂停

In [16]:
avg_coro.send(
    2023)  # 计算移动平均值：发送1个值，激活协程，协程将接收到的值（2023）赋值给num，并更新total、count、avg变量的值，同时开始循环的下一次迭代、产出avg变量的值、并等待下一次为num赋值

2023.0

In [17]:
avg_coro.send(1128)  # 计算移动平均值

1575.5

In [18]:
avg_coro.send(1410)  # 计算移动平均值

1520.3333333333333

# 4.使用装饰器预激活协程

In [19]:
# 使用预激装饰器简化协程
def prime_coroutine(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        gen = func(*args, **kwargs)
        gen.send(None)  # 预激协程
        return gen  # 返回激活后的协程对象

    return wrapper


@prime_coroutine
def make_avg():
    total = 0
    count = 0
    avg = None
    while True:
        num = yield avg
        total += num
        count += 1
        avg = total / count


avg_coro = make_avg()
avg_coro

<generator object make_avg at 0x000001159CC09BA0>

In [20]:
avg_coro.send(2023)  # 省去手动预激协程的步骤

2023.0

In [21]:
avg_coro.send(1128)

1575.5

In [22]:
avg_coro.send(1410)  # 与前面效果一样

1520.3333333333333

In [23]:
getgeneratorstate(avg_coro)  # 暂停状态

'GEN_SUSPENDED'

# 5.停止协程与异常处理

In [24]:
def make_avg():
    total = 0
    count = 0
    avg = None
    while True:
        num = yield avg
        total += num
        count += 1
        avg = total / count


avg_coro = make_avg()
avg_coro.send(None)
avg_coro.send(1128)

1128.0

In [25]:
# 协程中未处理的异常会向上抛出，传给send方法或next函数的调用方（即触发协程的对象）
avg_coro.send('1645')  # 发送给协程的不是数值、无法进行数值计算，因此协程内部会抛出异常

TypeError: unsupported operand type(s) for +=: 'int' and 'str'

In [26]:
avg_coro.send(1645)  # 协程内没有处理异常，协程会终止，如果重新激活协程或向协程发送数据，会抛出StopIteration异常->终止协程的一种方式：发送某个哨符值（None和Ellipsis等常量），让协程退出

StopIteration: 

In [27]:
# close方法显式发送异常给协程
# 自定义异常
class CustomException(Exception):
    pass


def handle_exception():
    print('coroutine starting...')
    while True:
        try:
            x = yield
        except CustomException:
            print('CustomException handled, continuing...')
        else:  # 没有异常时，显示接收到的值
            print(f'coroutine received: {x}')
    raise RuntimeError('Never happen!!!')  # 永远不会执行：因为只有未处理的异常才会中止无限循环，而一旦出现未处理的异常，协程会立即终止


exc_coro = handle_exception()
exc_coro.send(None)

coroutine starting...


In [28]:
exc_coro.send(1128)

coroutine received: 1128


In [29]:
# 调用close方法关闭协程
exc_coro.close()  # 要想退出无限循环、终止协程，可以调用close方法，也可以调用send方法传入None（即avg_coro.send(None)），但是此时应该在协程定义内部对send的值进行判断，如果是数值型才继续计算，否则应通过退出循环或主动抛出StopIteration异常的方式来终止协程
getgeneratorstate(exc_coro)  # # 结束状态，无法再继续接收值

'GEN_CLOSED'

In [30]:
exc_coro.send(1920)  # 协程处于结束状态、无法再继续接收值，抛出StopIteration异常

StopIteration: 

In [31]:
exc_coro = handle_exception()
exc_coro.send(None)
exc_coro.send(1128)
exc_coro.throw(CustomException)  # 传入协程可以处理的异常，会在协程内部捕获异常，并继续向前执行到下一个yield表达式，而产出的值会成为生成器调用throw方法得到的返回值，即下一个yield表达式的值

coroutine starting...
coroutine received: 1128
CustomException handled, continuing...


In [32]:
getgeneratorstate(exc_coro)  # 协程处于暂停状态，可以继续执行

'GEN_SUSPENDED'

In [33]:
exc_coro.throw(ZeroDivisionError)  # 传入协程无法处理的异常，协程内部无法捕获，会终止协程，并向上抛出异常、传到调用方

ZeroDivisionError: 

In [34]:
getgeneratorstate(exc_coro)  # 结束状态

'GEN_CLOSED'

# 6.协程返回值

In [35]:
# 计算移动平均值，使得协程具有返回值
Result = namedtuple('Result', 'count average')  # 使用namedtuple创建了一个名为Result的结果对象，有两个属性count和average


def make_avg():
    total = 0
    count = 0
    avg = None
    while True:
        num = yield
        if num is None:  # 在接收到None时中断循环，来终止协程，并得到返回值，同时生成器对象会抛出StopIteration异常
            break

        total += num
        count += 1
        avg = total / count
    return Result(count, avg)  # 异常对象的value属性保存着返回值


avg_coro = make_avg()
avg_coro.send(None)
avg_coro.send(1128)
avg_coro.send(2006)

In [36]:
# 获取结果
try:
    avg_coro.send(None)
except StopIteration as e:  # 捕获StopIteration异常，即可获取make_avg函数返回的值
    result = e.value  # return表达式的值会偷偷传给调用方，赋值给StopIteration异常的value属性，看似不合常理，但是能保留生成器对象的常规行为——耗尽时抛出StopIteration异常，这是PEP380约定的方式，因此yield from结构需要在内部自动捕获StopIteration异常
result

Result(count=2, average=1567.0)

# 7.协程的优势
协程是一种用户态的轻量级线程，本质上是单线程，拥有自己的寄存器上下文和栈，所以能保留上一次调用时的状态，每次过程重入时，就相当于进入上一次调用的状态。
与多进程相比，无需进程上下文切换的开销；与多线程相比，无需使用多线程的锁机制。执行效率要高于多进程和多线程。

以生产者-消费者模型为例：
在多线程的方式下，会创建2个线程，一个线程给生产者，一个线程给消费者，2个线程间通过同步机制来控制队列，从而获得数据，这一不小心就会死锁，虽然在Python中，Python自带的队列已经是线程安全的（即不会死锁）；
如果是使用协程来实现，可以用yield关键字轻松实现高效的生产者-消费者模型，具体而言：生产者生成数据后，直接通过send方法跳转到消费者并开始执行，消费者执行完后，再通过
yield关键字切换回生产者，效率很高。

In [37]:
# 协程实现生产者-消费者模型
def consumer():
    res = ''
    while True:
        n = yield res
        if not n:
            return
        print(f'[消费者]消费数据{n}...')
        res = '200 OK'


def producer(c):
    c.send(None)  # 激活消费者
    n = 0
    while n < 10:
        n += 1
        print(f'[生产者]生产数据{n}...')
        res = c.send(n)
        print(f'[生产者]消费者返回值：{res}')
        time.sleep(random.random() * 5)
    c.close()


c = consumer()
producer(c)

[生产者]生产数据1...
[消费者]消费数据1...
[生产者]消费者返回值：200 OK
[生产者]生产数据2...
[消费者]消费数据2...
[生产者]消费者返回值：200 OK
[生产者]生产数据3...
[消费者]消费数据3...
[生产者]消费者返回值：200 OK
[生产者]生产数据4...
[消费者]消费数据4...
[生产者]消费者返回值：200 OK
[生产者]生产数据5...
[消费者]消费数据5...
[生产者]消费者返回值：200 OK
[生产者]生产数据6...
[消费者]消费数据6...
[生产者]消费者返回值：200 OK
[生产者]生产数据7...
[消费者]消费数据7...
[生产者]消费者返回值：200 OK
[生产者]生产数据8...
[消费者]消费数据8...
[生产者]消费者返回值：200 OK
[生产者]生产数据9...
[消费者]消费数据9...
[生产者]消费者返回值：200 OK
[生产者]生产数据10...
[消费者]消费数据10...
[生产者]消费者返回值：200 OK


# 8.yield from
在生成器中使用yield from时，子生成器会获得控制权，把产出的值传给生成器的调用方，即调用方可以直接控制子生成器。yield from的主要功能是打开双向通道，把最外层的调用方与最内层的子生成器连接起来，这样二者可以直接发送和产出值，还可以直接传入异常，而不用在位于中间的协程中添加大量处理异常的代码。

因为委派生成器相当于管道，所以可以把任意数量个委派生成器连接在一起：一个委派生成器使用yield from调用一个子生成器，而那个子生成器本身也是委派生成器，使用yield from调用另一个子生成器，以此类推。这个链条最终要以一个只使用yield表达式的简单生成器结束，或者以任何可迭代的对象结束（即可迭代对象正常迭代完）。同时，任何yield from链条都必须由客户驱动，在最外层委派生成器上调用next函数或send方法，也可以隐式调用，例如使用for循环。

In [38]:
# 分析yield from对代码执行流程与数据流动的影响
def make_avg():
    '''子生成器：从yield from获取任务并完成具体实现的生成器，用于计算移动平均值'''
    total = 0
    count = 0
    avg = None
    while True:
        num = yield  # 客户端发送的各个值都赋值给num
        if num is None:  # 终止条件：在接收到None时中断循环，来终止协程，并得到返回值；如果不终止，则上层的生成器（使用yield from表达式调用本协程的生成器）会永远阻塞
            break

        total += num
        count += 1
        avg = total / count
    return Result(count, avg)  # 子生成器可以执行return语句返回一个值，而这个返回的值会成为yield from表达式的值，因此返回值Result会成为grouper函数中yield from表达式的值


def grouper(results, key):
    '''委派生成器：包含有 yield from表达式的生成器函数，负责给子生成器委派任务'''
    while True:  # 循环每次迭代时会新建一个make_avg实例，每个实例都是作为协程使用的生成器对象
        results[key] = yield from make_avg()  # grouper发送的每个值都会经由yield from处理，通过管道传给make_avg实例；grouper会在yield from处暂停，等待make_avg实例处理客户端发来的值


def client(data):
    '''调用方：指调用委派生成器的客户端'''
    results = {}
    for key, values in data.items():
        group = grouper(results,
                        key)  # group是调用grouper函数得到的生成器对象，传给grouper函数的参数results用于收集结果、key是数据中具体的键，group作为协程使用；新建grouper实例后，前一个grouper实例（以及它创建的尚未终止的make_avg子生成器实例）会被垃圾回收机制自动回收
        group.send(None)  # 预激协程，进入while True循环，调用子生成器make_avg后，在yield from表达式处暂停
        for value in values:
            group.send(
                value)  # 将各个value传给grouper，传入的值最终到达make_avg函数中的num = yield，将传入的值赋值给变量num用于计算均值，而group不知道传入的值是什么、并在yield from处暂停
        # 内层循环结束后，group实例依旧在yield from处暂停，grouper中为results[key]赋值的语句还没有执行
        group.send(
            None)  # 调用send方法传入None终止当前的make_avg实例，也让grouper继续运行，再创建下一个make_avg实例、处理下一组值；如果没有调用group.send(None)，则make_avg子生成器永远不会终止，永远不会为results[key]赋值，委派生成器group也永远不会再次激活

    # 输出结果
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print(f'{result.count} {group} averaging {result.average:.2f} {unit}')


data = {
    'girls;m': [1.6, 1.51, 1.25, 1.3, 1.41, 1.37, 1.43, 1.46, 1.25, 1.43],
    'boys;m': [1.88, 1.5, 1.32, 1.25, 1.37, 1.58, 1.35, 1.49, 1.36],
    'girls;kg': [40.9, 48.5, 42.3, 42.2, 45.2, 41.7, 46.5, 38.0, 47.6, 34.8],
    'boys;kg': [59.0, 50.8, 43.2, 39.8, 43.1, 48.6, 41.6, 40.6, 56.3]
}
client(data)

9 boys averaging 47.00 kg
9 boys averaging 1.46 m
10 girls averaging 42.77 kg
10 girls averaging 1.40 m


# 9.过时协程用法：asyncio与yield from结合
asyncio是Python 3.4 试验性引入的异步I/O框架，提供了基于协程做异步I/O编写单线程并发代码的基础设施，其核心组件有事件循环（Event Loop）、协程（Coroutine）、任务（Task）、未来对象（Future）以及其他一些扩充和辅助性质的模块。其中，在事件循环中，要执行的逻辑其实是一个个的事件，这些事件存放在事件队列中，被主线程一个个循环处理，当一个事件中存在耗时任务时，会将该事件交由操作系统去执行，并告知操作系统，执行完后回调提供的方法，当操作系统执行完后，会将回调方法放回事件循环队列中，等等主线程循环处理。asyncio刚引入时，需要与yield from结合使用，现在已经成为过时的实现方式。

In [39]:
# 同步任务
def sync_task(n):
    print(f'sync task {n} start...')
    time.sleep(n)
    print(f'sync task finished, cost {n} seconds')


start = time.time()
sync_task(1)
sync_task(2)
sync_task(3)
print(f'sync task cost {time.time() - start:.2f} seconds')  # 同步的方式实现多个任务的时间是分别执行这些任务时间的总和

sync task 1 start...
sync task finished, cost 1 seconds
sync task 2 start...
sync task finished, cost 2 seconds
sync task 3 start...
sync task finished, cost 3 seconds
sync task cost 6.04 seconds


In [40]:
# 使用yield from与asyncio将同步方式改成异步
@asyncio.coroutine  # 协程函数的标志：需要在每一个任务函数前加装饰器asyncio.coroutine，并在函数中使用yield from
def async_task(n):
    print(f'async task {n} start...')
    yield from asyncio.sleep(n)  # 异步等待n秒
    print(f'async task finished, cost {n} seconds')
    return async_task.__name__ + str(n)


def client():
    '''调用方'''
    tasks = [async_task(1), async_task(2), async_task(3)]  # 3.把多个任务放到任务列表中，打包任务
    done, pending = yield from asyncio.wait(tasks)  # 4.子生成器：使用asyncio.wait(tasks)来获取一个awaitable objects即可等待对象的集合，通过yield from返回一个包含(done, pending)的元组，done表示已完成的任务列表、pending表示未完成的任务列表

    for task in done:  # 5.遍历done列表，逐个调用result()取出结果
        print(f'async task {task.result()} finished')


start = time.time()
loop = asyncio.get_event_loop()  # 1.创建事件循环对象
try:
    loop.run_until_complete(client())  # 2.运行协程，把调用方协程作为参数，调用方负责调用其他委托生成器，直到所有任务完成
finally:
    loop.close()  # 6.关闭事件循环
print(f'all tasks finished, cost {time.time() - start:.2f} seconds')  # 异步的方式实现多个任务的时间是最长的任务执行时间

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24796\4103844703.py:3: DeprecationWarning: "@coroutine" decorator is deprecated since Python 3.8, use "async def" instead
  def async_task(n):


async task 2 start...
async task 3 start...
async task 1 start...
async task finished, cost 1 seconds
async task finished, cost 2 seconds
async task finished, cost 3 seconds
async task async_task2 finished
async task async_task3 finished
async task async_task1 finished
all tasks finished, cost 3.00 seconds


上面的代码只支持Python 3.6及以下、Python 3.4及以上的版本，大于等于3.7的版本已弃用，所以会报错，其中在Jupyter Notebook中报错`RuntimeError: Cannot close a running event loop`，在Python脚本中报错`TypeError: cannot 'yield from' a coroutine object in a non-coroutine generator`。

通过使用协程，极大提高了多任务的执行效率，程序最后消耗的时间是任务队列中耗时最多时间任务的时长。

# 10.新的协程用法：async与await
asyncio模块结合yield from实现异步协程的语法不够简洁，与生成器、装饰器交织在一起，容易让人混淆。为了语法更加简洁。Python3.5新增了async、await语法来实现异步协程。
其中：
- async/await：python3.5之后用于定义协程的关键字，async定义一个协程、await用于挂起阻塞的异步调用接口。
- event_loop：事件循环，程序开启一个无限的循环，可以把一些函数注册到事件循环上，当满足事件发生的时候，调用相应的协程函数。协程函数不能直接调用运行，需要将协程注册到事件循环，并启动事件循环才能使用。
- coroutine ：协程对象，即一个使用async关键字定义的函数，它的调用不会立即执行函数，而是会返回一个协程对象，协程对象需要注册到事件循环，由事件循环调用。
- task：任务，是对协程的进一步封装，包含了任务的各种状态。协程对象不能直接运行，在注册事件循环的时候，其实是run_until_complete方法将协程自动包装成任务（task）对象。
- future： 代表将来执行或没有执行的任务的结果，在本质上和task没有区别。

async/await与yield from风格的协程底层实现方式相同，同时async/await风格的代码隐藏了装饰器、yield from语法，方便人们的理解，同时也让代码更加简洁。

In [41]:
# 1.创建协程
async def async_task(n):  # 修改1：在def前用关键字async声明，即可定义协程函数
    '''修改使用yield from与asyncio实现异步为async和await实现'''
    print(f'async task {n} start...')
    await asyncio.sleep(n)  # 修改2：await关键字，协程遇到await，事件循环将会挂起该协程、执行别的协程，直到其他的协程也挂起或者执行完毕，再进行下一个协程的执行
    print(f'async task finished, cost {n} seconds')
    return async_task.__name__ + str(n)


coro = async_task(2)  # 调用协程函数得到协程对象，此时协程对象还没有执行
loop = asyncio.get_event_loop()  # 创建事件循环
coro  # coroutine对象

<coroutine object async_task at 0x000001159E7BEE40>

In [42]:
# 注册协程对象到事件循环，并启动
loop.run_until_complete(coro)  # Jupyter中会报错RuntimeError: This event loop is already running，需要在Python脚本中运行

async task 2 start...
async task finished, cost 2 seconds

In [43]:
# 2.任务对象task：手动将协程包装成任务
# 方式1：使用create_task()创建task
coro = async_task(3)
task = loop.create_task(coro)
task  # Task对象：未将task添加到事件循环之前，状态是pending

<Task pending name='Task-5' coro=<async_task() running at C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24796\1366162558.py:2>>

In [44]:
loop.run_until_complete(task)
task  # Task对象：task对象执行完毕后，状态是finished，result属性保存了协程函数的返回值

async task 3 start...
async task finished, cost 3 seconds

<Task finished name='Task-5' coro=<async_task() done, defined at C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16424\1366162558.py:2> result='async_task3'>

In [45]:
# 方式2：使用asyncio的ensure_future()方法创建task
coro = async_task(3)
task = asyncio.ensure_future(
    coro)  # 使用asyncio的ensure_future()方法创建task，并将coroutine对象转化成task对象；可以在loop未定义前创建task，效果与使用create_task()创建task相同
task

<Task pending name='Task-6' coro=<async_task() running at C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24796\1366162558.py:2>>

In [46]:
loop.run_until_complete(task)
task

async task 3 start...
async task finished, cost 3 seconds

<Task finished name='Task-6' coro=<async_task() done, defined at C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24796\1366162558.py:2> result='async_task3'>

In [47]:
# 3.绑定回调函数
def callback(task):  # 自定义回调函数，输出task的结果
    print(f'task result: {task.result()}')


coro = async_task(4)
task = loop.create_task(coro)
task.add_done_callback(callback)  # 绑定回调函数
task

<Task pending name='Task-7' coro=<async_task() running at C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24796\1366162558.py:2> cb=[callback() at C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24796\3686030089.py:2]>

In [48]:
loop.run_until_complete(task)
task

async task 4 start...
async task finished, cost 4 seconds
task result: async_task4

<Task finished name='Task-7' coro=<async_task() done, defined at C:\Users\LENOVO\AppData\Local\Temp\ipykernel_24796\1366162558.py:2> result='async_task4'>

In [49]:
# 4.多任务协程：需要执行多个任务时，可以定义一个任务列表，并将需要完成的协程任务都加进去，执行loop.run_until_complete(asyncio.wait(tasks))，适用于网络请求、文件读取等耗时任务，同时需要使用await关键字将耗时的操作挂起、让函数让出控制权。
start = time.time()
tasks = [async_task(1), async_task(2), async_task(3)]
loop.run_until_complete(asyncio.wait(tasks))  # 完成事件循环，直到所有任务执行完成
print(f'all tasks finished, cost {time.time() - start:.2f} seconds')  # 与前面使用yield from与asyncio效果相同

async task 2 start...
async task 1 start...
async task 3 start...
async task finished, cost 1 seconds
async task finished, cost 2 seconds
async task finished, cost 3 seconds
all tasks finished, cost 3.01 seconds

In [50]:
# 使用asyncio.run方法（Python 3.7及以后）：省去显式定义事件循环的步骤
asyncio.run(async_task(3))

async task 3 start...
async task finished, cost 3 seconds

在Python中，协程（coroutine）和异步（async）是两个相关的概念，它们都可以用于实现非阻塞性的并发操作。但是，它们之间也有一些重要的区别：

1. **协程（Coroutine）**：协程是一种用户态的轻量级线程，它不需要操作系统的调度机制，从Python的实现角度来看，协程是一种特殊类型的函数，可以在执行过程中挂起并恢复，允许在函数内部进行多任务处理。协程可以通过`async def`来定义，并使用`await`来挂起和恢复。在Python 3.4中引入的`asyncio`库提供了对协程的原生支持。
2. **异步（Async）**：异步编程是一种并发模型，其中操作以非阻塞的方式执行，允许其他任务在等待操作完成时继续执行。在Python中，可以通过`asyncio`库和`async/await`语法来实现异步编程。

从`yield`、`yield from`、`asyncio`和`async/await`角度解释协程和异步的关系：

1. **yield**：在Python中，`yield`关键字用于定义生成器函数。生成器是一种特殊类型的迭代器，可以用于按需生成数据。使用`yield`可以将函数的执行暂停在某个点，并保存内部状态，以便下次调用时可以从上次暂停的位置继续执行。
2. **yield from**：在Python 3.3中引入的`yield from`语法允许在一个生成器内部直接使用另一个生成器。通过使用`yield from`，可以将一个生成器的值生成器传递给另一个生成器，并在需要时暂停和恢复执行。
3. **asyncio**：`asyncio`是Python标准库的一部分，提供了对异步I/O、协程和事件循环的支持。通过使用`asyncio`库，可以编写异步代码，并利用Python的协程和生成器功能来处理并发操作。
4. **async/await**：在Python 3.5中引入的`async`和`await`关键字使得编写异步代码更加方便。通过使用`async def`定义异步函数，并使用`await`关键字挂起和恢复执行，可以编写清晰、易读的异步代码。

综上所述，协程和异步是相互关联的概念，它们都可以用于实现非阻塞性的并发操作：协程是一种函数类型，可以通过挂起和恢复执行来处理多个任务；而异步编程是一种并发模型，它利用事件循环和异步I/O来执行非阻塞操作。在Python中，可以使用`asyncio`库和`async/await`语法来简化异步编程。